In [1]:
import swe_bench_constants
from test_spec import make_test_spec, get_test_specs_from_dataset, TestSpec
from datasets import load_dataset
import subprocess
import os
dataset = load_dataset('princeton-nlp/SWE-bench_Verified', split='test')
def get_spec(sbi: swe_bench_constants.SWEbenchInstance) -> swe_bench_constants.SpecInstance:
    return swe_bench_constants.MAP_REPO_VERSION_TO_SPECS[sbi['repo']][sbi['version']]
def get_data(i: int) -> tuple[swe_bench_constants.SWEbenchInstance, swe_bench_constants.SpecInstance]:
    return dataset[i], get_spec(dataset[i])

In [2]:
get_data(1)

({'repo': 'astropy/astropy',
  'instance_id': 'astropy__astropy-13033',
  'base_commit': '298ccb478e6bf092953bca67a3d29dc6c35f6752',
  'patch': 'diff --git a/astropy/timeseries/core.py b/astropy/timeseries/core.py\n--- a/astropy/timeseries/core.py\n+++ b/astropy/timeseries/core.py\n@@ -55,6 +55,13 @@ class BaseTimeSeries(QTable):\n     _required_columns_relax = False\n \n     def _check_required_columns(self):\n+        def as_scalar_or_list_str(obj):\n+            if not hasattr(obj, "__len__"):\n+                return f"\'{obj}\'"\n+            elif len(obj) == 1:\n+                return f"\'{obj[0]}\'"\n+            else:\n+                return str(obj)\n \n         if not self._required_columns_enabled:\n             return\n@@ -76,9 +83,10 @@ def _check_required_columns(self):\n \n             elif self.colnames[:len(required_columns)] != required_columns:\n \n-                raise ValueError("{} object is invalid - expected \'{}\' "\n-                                 "as the

In [3]:
d_specs = get_test_specs_from_dataset("./foo", dataset)

In [4]:
d = d_specs[10]

In [5]:
for l in d.repo_script_list:
    print(l)

git clone -o origin https://github.com/astropy/astropy ./foo/astropy__astropy-14365
chmod -R 777 ./foo/astropy__astropy-14365
cd ./foo/astropy__astropy-14365
git reset --hard 7269fa3e33e8d02485a647da91a5a2a60a06af61
git remote remove origin
source /opt/miniconda3/bin/activate
conda activate astropy__astropy-14365
echo "Current environment: $CONDA_DEFAULT_ENV"
sed -i 's/requires = \["setuptools",/requires = \["setuptools==68.0.0",/' pyproject.toml
python -m pip install -e .[test] --verbose


In [6]:
for l in d.eval_script_list:
    print(l)

source /opt/miniconda3/bin/activate
conda activate astropy__astropy-14365
cd ./foo/astropy__astropy-14365
git config --global --add safe.directory ./foo/astropy__astropy-14365
cd ./foo/astropy__astropy-14365
git status
git show
git diff 7269fa3e33e8d02485a647da91a5a2a60a06af61
source /opt/miniconda3/bin/activate
conda activate astropy__astropy-14365
python -m pip install -e .[test] --verbose
git checkout 7269fa3e33e8d02485a647da91a5a2a60a06af61 astropy/io/ascii/tests/test_qdp.py
git apply -v - <<'EOF_114329324912'
diff --git a/astropy/io/ascii/tests/test_qdp.py b/astropy/io/ascii/tests/test_qdp.py
--- a/astropy/io/ascii/tests/test_qdp.py
+++ b/astropy/io/ascii/tests/test_qdp.py
@@ -43,7 +43,18 @@ def test_get_tables_from_qdp_file(tmp_path):
     assert np.isclose(table2["MJD_nerr"][0], -2.37847222222222e-05)
 
 
-def test_roundtrip(tmp_path):
+def lowercase_header(value):
+    """Make every non-comment line lower case."""
+    lines = []
+    for line in value.splitlines():
+        if

In [7]:
for l in d.env_script_list:
    print(l)

source /opt/miniconda3/bin/activate
conda create -n astropy__astropy-14365 python=3.9  -y
conda activate astropy__astropy-14365
python -m pip install attrs==23.1.0 exceptiongroup==1.1.3 execnet==2.0.2 hypothesis==6.82.6 iniconfig==2.0.0 numpy==1.25.2 packaging==23.1 pluggy==1.3.0 psutil==5.9.5 pyerfa==2.0.0.3 pytest-arraydiff==0.5.0 pytest-astropy-header==0.2.2 pytest-astropy==0.10.0 pytest-cov==4.1.0 pytest-doctestplus==1.0.0 pytest-filter-subpackage==0.1.2 pytest-mock==3.11.1 pytest-openfiles==0.5.0 pytest-remotedata==0.4.0 pytest-xdist==3.3.1 pytest==7.4.0 PyYAML==6.0.1 setuptools==68.0.0 sortedcontainers==2.4.0 tomli==2.0.1


In [8]:
print(d.eval_script)

#!/bin/bash
set -uxo pipefail
source /opt/miniconda3/bin/activate
conda activate astropy__astropy-14365
cd ./foo/astropy__astropy-14365
git config --global --add safe.directory ./foo/astropy__astropy-14365
cd ./foo/astropy__astropy-14365
git status
git show
git diff 7269fa3e33e8d02485a647da91a5a2a60a06af61
source /opt/miniconda3/bin/activate
conda activate astropy__astropy-14365
python -m pip install -e .[test] --verbose
git checkout 7269fa3e33e8d02485a647da91a5a2a60a06af61 astropy/io/ascii/tests/test_qdp.py
git apply -v - <<'EOF_114329324912'
diff --git a/astropy/io/ascii/tests/test_qdp.py b/astropy/io/ascii/tests/test_qdp.py
--- a/astropy/io/ascii/tests/test_qdp.py
+++ b/astropy/io/ascii/tests/test_qdp.py
@@ -43,7 +43,18 @@ def test_get_tables_from_qdp_file(tmp_path):
     assert np.isclose(table2["MJD_nerr"][0], -2.37847222222222e-05)
 
 
-def test_roundtrip(tmp_path):
+def lowercase_header(value):
+    """Make every non-comment line lower case."""
+    lines = []
+    for line in v

In [9]:
# def setup_testspec(ts: TestSpec):
#     # execute each command in env_script_list
#     for l in ts.env_script_list:
#         subprocess.run(l, shell=True)
#     # execute each command in repo_script_list
#     for l in ts.repo_script_list:
#         subprocess.run(l, shell=True)

In [10]:
d_verified = load_dataset('princeton-nlp/SWE-bench_Verified', split='test')
d_lite = load_dataset('princeton-nlp/SWE-bench_Lite', split='test')

In [11]:
# intersection of verified and lite datasets
d_verified_dict = {}
for v in d_verified:
    d_verified_dict[v['instance_id']] = v
for l in d_lite:
    if l['instance_id'] in d_verified_dict:
        print(l['instance_id'])
        print(d_verified_dict[l['instance_id']])

astropy__astropy-12907
{'repo': 'astropy/astropy', 'instance_id': 'astropy__astropy-12907', 'base_commit': 'd16bfe05a744909de4b27f5875fe0d4ed41ce607', 'patch': "diff --git a/astropy/modeling/separable.py b/astropy/modeling/separable.py\n--- a/astropy/modeling/separable.py\n+++ b/astropy/modeling/separable.py\n@@ -242,7 +242,7 @@ def _cstack(left, right):\n         cright = _coord_matrix(right, 'right', noutp)\n     else:\n         cright = np.zeros((noutp, right.shape[1]))\n-        cright[-right.shape[0]:, -right.shape[1]:] = 1\n+        cright[-right.shape[0]:, -right.shape[1]:] = right\n \n     return np.hstack([cleft, cright])\n \n", 'test_patch': "diff --git a/astropy/modeling/tests/test_separable.py b/astropy/modeling/tests/test_separable.py\n--- a/astropy/modeling/tests/test_separable.py\n+++ b/astropy/modeling/tests/test_separable.py\n@@ -28,6 +28,13 @@\n p1 = models.Polynomial1D(1, name='p1')\n \n \n+cm_4d_expected = (np.array([False, False, True, True]),\n+                  n

In [12]:
d_verified[0]

{'repo': 'astropy/astropy',
 'instance_id': 'astropy__astropy-12907',
 'base_commit': 'd16bfe05a744909de4b27f5875fe0d4ed41ce607',
 'patch': "diff --git a/astropy/modeling/separable.py b/astropy/modeling/separable.py\n--- a/astropy/modeling/separable.py\n+++ b/astropy/modeling/separable.py\n@@ -242,7 +242,7 @@ def _cstack(left, right):\n         cright = _coord_matrix(right, 'right', noutp)\n     else:\n         cright = np.zeros((noutp, right.shape[1]))\n-        cright[-right.shape[0]:, -right.shape[1]:] = 1\n+        cright[-right.shape[0]:, -right.shape[1]:] = right\n \n     return np.hstack([cleft, cright])\n \n",
 'test_patch': "diff --git a/astropy/modeling/tests/test_separable.py b/astropy/modeling/tests/test_separable.py\n--- a/astropy/modeling/tests/test_separable.py\n+++ b/astropy/modeling/tests/test_separable.py\n@@ -28,6 +28,13 @@\n p1 = models.Polynomial1D(1, name='p1')\n \n \n+cm_4d_expected = (np.array([False, False, True, True]),\n+                  np.array([[True,  Tr

In [13]:
d_lite[0]

{'repo': 'astropy/astropy',
 'instance_id': 'astropy__astropy-12907',
 'base_commit': 'd16bfe05a744909de4b27f5875fe0d4ed41ce607',
 'patch': "diff --git a/astropy/modeling/separable.py b/astropy/modeling/separable.py\n--- a/astropy/modeling/separable.py\n+++ b/astropy/modeling/separable.py\n@@ -242,7 +242,7 @@ def _cstack(left, right):\n         cright = _coord_matrix(right, 'right', noutp)\n     else:\n         cright = np.zeros((noutp, right.shape[1]))\n-        cright[-right.shape[0]:, -right.shape[1]:] = 1\n+        cright[-right.shape[0]:, -right.shape[1]:] = right\n \n     return np.hstack([cleft, cright])\n \n",
 'test_patch': "diff --git a/astropy/modeling/tests/test_separable.py b/astropy/modeling/tests/test_separable.py\n--- a/astropy/modeling/tests/test_separable.py\n+++ b/astropy/modeling/tests/test_separable.py\n@@ -28,6 +28,13 @@\n p1 = models.Polynomial1D(1, name='p1')\n \n \n+cm_4d_expected = (np.array([False, False, True, True]),\n+                  np.array([[True,  Tr

In [14]:
print(d_lite[0]['test_patch'])

diff --git a/astropy/modeling/tests/test_separable.py b/astropy/modeling/tests/test_separable.py
--- a/astropy/modeling/tests/test_separable.py
+++ b/astropy/modeling/tests/test_separable.py
@@ -28,6 +28,13 @@
 p1 = models.Polynomial1D(1, name='p1')
 
 
+cm_4d_expected = (np.array([False, False, True, True]),
+                  np.array([[True,  True,  False, False],
+                            [True,  True,  False, False],
+                            [False, False, True,  False],
+                            [False, False, False, True]]))
+
+
 compound_models = {
     'cm1': (map3 & sh1 | rot & sh1 | sh1 & sh2 & sh1,
             (np.array([False, False, True]),
@@ -52,7 +59,17 @@
     'cm7': (map2 | p2 & sh1,
             (np.array([False, True]),
              np.array([[True, False], [False, True]]))
-            )
+            ),
+    'cm8': (rot & (sh1 & sh2), cm_4d_expected),
+    'cm9': (rot & sh1 & sh2, cm_4d_expected),
+    'cm10': ((rot & sh1) & sh2, cm_4d_expected),
+    

In [18]:
q = d_verified_dict['django__django-10914']

print("================== problem_statement ==================")
print(q['problem_statement'])
print("================== patch ==================")
print(q['patch'])
print("================== test_patch ==================")
print(q['test_patch'])

================== problem_statement ==================
Set default FILE_UPLOAD_PERMISSION to 0o644.
Description
	
Hello,
As far as I can see, the ​File Uploads documentation page does not mention any permission issues.
What I would like to see is a warning that in absence of explicitly configured FILE_UPLOAD_PERMISSIONS, the permissions for a file uploaded to FileSystemStorage might not be consistent depending on whether a MemoryUploadedFile or a TemporaryUploadedFile was used for temporary storage of the uploaded data (which, with the default FILE_UPLOAD_HANDLERS, in turn depends on the uploaded data size).
The tempfile.NamedTemporaryFile + os.rename sequence causes the resulting file permissions to be 0o0600 on some systems (I experience it here on CentOS 7.4.1708 and Python 3.6.5). In all probability, the implementation of Python's built-in tempfile module explicitly sets such permissions for temporary files due to security considerations.
I found mentions of this issue ​on GitHub,